In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import gzip
from collections import defaultdict
import csv
import random
import numpy
import scipy.optimize
f=gzip.open('amazon_reviews_us_Gift_Card_v1_00.tsv.gz','rt') 
header=f.readline().strip().split("\t")
header

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [6]:
dataset=[]
for line in f:
  fields=line.strip().split('\t')
  d=dict(zip(header,fields))
  d['star_rating']=int(d['star_rating'])
  d['helpful_votes']=int(d['helpful_votes'])
  d['total_votes']=int(d['total_votes'])
  dataset.append(d)
len(dataset)

149086

In [7]:
dataset=[d for d in dataset if 'review_date' in d ]
print('\n')
len(dataset)

149086

In [8]:
#useful data structures
usersperitem=defaultdict(set)
itemsperuser=defaultdict(set)
itemnames={}

In [9]:
for d in dataset:
  user,item=d['customer_id'],d['product_id']
  usersperitem[item].add(user)
  itemsperuser[user].add(item)
  itemnames[item]=d['product_title']

In [10]:
import itertools
# Get first N items in dictionary  
out = dict(itertools.islice(usersperitem.items(), 1)) 
# printing result   
print("Dictionary limited by 1 elements is : " + str(out))

Dictionary limited by 1 elements is : {'B004LLIL5A': {'51145759', '22180981', '20441276', '2226191', '1101127', '26445564', '50856530', '51759166', '13843302', '924745', '2132076', '36976120', '39884181', '20749172', '40734489', '41917571', '16128751', '5399612', '27917871', '15777597', '39069259', '16038587', '20766005', '50755849', '20661270', '43898445', '11435194', '37299793', '43139297', '23443462', '10726804', '28983094', '36755284', '45125372', '40659544', '37724184', '46128410', '14670707', '14434733', '26943573', '4950395', '18188059', '24430158', '46300743', '52166203', '39806590', '29013622', '40908384', '36302586', '51444591', '14219190', '19155989', '20207428', '35306384', '565538', '13290034', '41908756', '8885135', '13414452', '49146906', '29470291', '39846038', '22608306', '4876335', '24373155', '24104818', '5560462', '16554524', '32503939', '19851734', '45846477', '30603260', '30130961', '28796161', '34256590', '49624467', '47025706', '15268489', '52162922', '23384301'

In [11]:
'''we want a recommendation function that return items
similar to a caqndidate item i,our strategy is as follows:
find the set of users who purchased i
iterate over all other items other than i
for all other items compute their 
similiarity with i and store it.
sort all other items by jaccard similiarity
return the most similar.'''


'we want a recommendation function that return items\nsimilar to a caqndidate item i,our strategy is as follows:\nfind the set of users who purchased i\niterate over all other items other than i\nfor all other items compute their \nsimiliarity with i and store it.\nsort all other items by jaccard similiarity\nreturn the most similar.'

In [12]:
def jaccard(s1,s2):
  numer=len(s1.intersection(s2))
  denom=len(s1.union(s2))
  return(numer/denom)
def mostsimilar(i):
  similiarities=[]
  users=usersperitem[i]
  for l2 in usersperitem:
    if l2==i:
      continue
    sim=jaccard(users,usersperitem[l2])
    similiarities.append((sim,l2))
  return similiarities[:10]

In [13]:
query=dataset[2]['product_id']
query

'B00IX1I3G6'

In [14]:
mostsimilar(query)

[(0.0016906170752324597, 'B004LLIL5A'),
 (0.017129374272231414, 'B004LLIKVU'),
 (0.0, 'B005ESMGV4'),
 (0.0, 'B004KNWWU4'),
 (0.0, 'BT00CTP2EE'),
 (0.0006634232640424591, 'B004W8D102'),
 (0.000230361667818475, 'B00H5BNLUS'),
 (0.0, 'B004KNWX6C'),
 (0.000445632798573975, 'BT00CTOYC0'),
 (0.0008775008775008775, 'B00H5BMH44')]

In [15]:
itemnames[query]

'Amazon.com Gift Card Balance Reload'

In [16]:
[itemnames[x[1]] for x in mostsimilar(query)]

['Amazon eGift Card - Celebrate',
 'Amazon.com eGift Cards',
 'Amazon.com Gift Cards, Pack of 3 (Various Designs)',
 'Amazon Gift Card - Print - Happy Birthday (Birds)',
 'Amazon.com Gift Card in a Greeting Card (Various Designs)',
 'Amazon Gift Card - Print - Amazon Boxes',
 'Amazon eGift Card - Hoops and Yoyo Thank You Very Much (Animated) [Hallmark]',
 'Amazon Gift Card - Print - Celebrate',
 'Amazon.com $15 Gift Card in a Greeting Card (Amazon Surprise Box Design)',
 'Amazon eGift Card - Hoops and Yoyo Cake Face (Animated) [Hallmark]']

In [17]:
'''it is sufficient to iterate over thoose
items purchased by one of the users 
who purchased i.
find the set of users who purchased i.
iterate over all users who purchased i
build a condidate set from all items 
those users consumed.for items in this set,
compute their similiarity with i and store it.
sort all other items by jaccard similiarity
return the most similar'''


'it is sufficient to iterate over thoose\nitems purchased by one of the users \nwho purchased i.\nfind the set of users who purchased i.\niterate over all users who purchased i\nbuild a condidate set from all items \nthose users consumed.for items in this set,\ncompute their similiarity with i and store it.\nsort all other items by jaccard similiarity\nreturn the most similar'

In [18]:
def mostsimilarfast(i):
  similiarities=[]
  users=usersperitem[i]
  candidateitems=set()
  for u in users:
    candidateitems=candidateitems.union(itemsperuser[u])
  for l2 in candidateitems:
    if l2==i:
      continue
    sim=jaccard(users,usersperitem[l2])
    similiarities.append((sim,l2))
    similiarities.sort(reverse=True)
    return similiarities[:10]

In [19]:
query=dataset[2]['product_id']
mostsimilarfast(query)

[(0.0004463289444320464, 'B00A4EK4YO')]

In [20]:
'''The user(u)'s rating for an item i is a 
weighted combination of all of their
previous ratings for item j.
the weight for each rating is given by
the jaccard similiarity between i and j.''' 

"The user(u)'s rating for an item i is a \nweighted combination of all of their\nprevious ratings for item j.\nthe weight for each rating is given by\nthe jaccard similiarity between i and j."

In [21]:
#more utility dta structures
reviewsperuser=defaultdict(list)
reviewsperitem=defaultdict(list)
for d in dataset:
  user,item=d['customer_id'],d['product_id']
  reviewsperuser[user].append(d)
  reviewsperitem[item].append(d)

In [22]:
ratingmean=sum([d['star_rating'] for d in dataset])/len(dataset)
ratingmean

4.731363105858364

In [23]:
def predictrating(user,item):
  ratings=[]
  similiarities=[]
  for d in reviewsperuser[user]:
    i2=d['product_id']
    if i2==item:continue
    ratings.append(d['star_rating'])
    similiarities.append(d['star_rating'])
    similiarities.append(jaccard(usersperitem[item],usersperitem[i2]))
  if sum(similiarities)>0:
    weightedratings=[(x*y) for x,y in zip(ratings,similiarities)]
    return sum(weightedratings)/sum(similiarities)
  else:
    return ratingmean
    

In [24]:
dataset[1]

{'customer_id': '42489718',
 'helpful_votes': 0,
 'marketplace': 'US',
 'product_category': 'Gift Card',
 'product_id': 'B004LLIKVU',
 'product_parent': '473048287',
 'product_title': 'Amazon.com eGift Cards',
 'review_body': "It's an Amazon gift card and with over 982398324908 items I am sure you will find something you love.",
 'review_date': '2015-08-31',
 'review_headline': 'Gift card for the greatest selection of items online',
 'review_id': 'RJ7RSBCHUDNNE',
 'star_rating': 5,
 'total_votes': 0,
 'verified_purchase': 'Y',
 'vine': 'N'}

In [25]:
u,i=dataset[0]['customer_id'],dataset[0]['product_id']
predictrating(u,i)

4.731363105858364

In [26]:
def MSE(predictions,labels):
  differences=[(x-y)**2 for x,y in zip(predictions,labels)]
  return sum(differences)/len(differences)

In [27]:
alwayspredictmean=[ratingmean for d in dataset ]
cpredictions=[predictrating(d['customer_id'],d['product_id']) for d in dataset]

In [28]:
labels=[d['star_rating']for d in dataset]
MSE(alwayspredictmean,labels)

0.6877446063986206

In [29]:
labels=[d['star_rating']for d in dataset]
MSE(cpredictions,labels)

0.7314833254421422

In [30]:
#coding for latent factor models
N=len(dataset)
nusers=len(reviewsperuser)
nitems=len(reviewsperitem)
users=list(reviewsperuser.keys())
items=list(reviewsperitem.keys())

In [31]:
alpha=ratingmean
userbiases=defaultdict(float)
itembiases=defaultdict(float)

In [32]:
def prediction(user,item):
  return alpha+userbiases[user]+itembiases[item]

In [33]:
'''the gradient descent library we will use expects
a single vector of parameters(theta)which we have to unpack 
to produce alpha and beta'''

'the gradient descent library we will use expects\na single vector of parameters(theta)which we have to unpack \nto produce alpha and beta'

In [34]:
def unpack(theta):
  global alpha
  global userbiases
  global itembiases
  alpha=theta[0]
  userbiases=dict(zip(users,theta[1:nusers+1]))
  itembiases=dict(zip(items,theta[1+nusers:]))

In [35]:
'''the next function just implement the full cost function 
which is required by the gradient descent library'''
def cost(theta,lbels,lamb):
  unpack(theta)
  predictions=[prediction(d['customer_id'],d['product_id'])for d in dataset]
  cost=MSE(predictions,labels)
  print("MSE= "+str(cost))
  for u in userbiases:
    cost+=lamb*userbiases[u]**2
  for i in itembiases:
    cost+=lamb*itembiases[i]**2
  return(cost)
             

'the next function just implement the full cost function \nwhich is required by the gradient descent library'

In [36]:
'''next we implement the derivative function which has a 
corresponding derivative term for each parameter'''
def derivative(theta,labels,lamb):
  unpack(theta)
  N=len(dataset)
  dalpha=0
  dUserBiases=defaultdict(float)
  dItemBiases=defaultdict(float)
  for d in dataset:
    u,i=d['customer_id'],d['product_id']
    pred=prediction(u,i)
    diff=pred-d['star_rating']
    dalpha+=2/N*diff
    dUserBiases[u]+=2/N*diff
    dItemBiases[i]+=2/N*diff
  for u in userbiases:
    dUserBiases[u]+=2*lamb*userbiases[u]
  for i in itembiases:
    dItemBiases[i]=2*lamb*itembiases[i]
  dtheta=[dalpha]+[dUserBiases[u] for u in users]+[ dItemBiases[i] for i in items]
  return numpy.array(dtheta)

'next we implement the derivative function which has a \ncorresponding derivative term for each parameter'

In [37]:
MSE(alwayspredictmean,labels)

0.6877446063986206

In [38]:
'''the gradient descent library we use is called lbfgs
this is a general purpose gradient descent algorithm 
which simply requires that we provide a cost function f(x)
and a dervative function f'(x).it can be relatively
straightforwardly adapted to other gradient descent problems'''
scipy.optimize.fmin_l_bfgs_b(cost,[ratingmean]+[0.0]*(nusers+nitems),derivative,args=(labels,0.001))

"the gradient descent library we use is called lbfgs\nthis is a general purpose gradient descent algorithm \nwhich simply requires that we provide a cost function f(x)\nand a dervative function f'(x).it can be relatively\nstraightforwardly adapted to other gradient descent problems"

MSE= 0.6877446063986206
MSE= 0.6834014493675252
MSE= 0.682120809159982
MSE= 0.6823272091538863
MSE= 0.6780331803686451
MSE= 0.678383342887593


(array([4.73130139e+00, 1.78940106e-03, 1.78940106e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 0.6830471082405756,
 {'funcalls': 6,
  'grad': array([-2.91696431e-07, -1.80519386e-09, -1.80519386e-09, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
  'nit': 4,
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0})

In [39]:
#implementing a latent factor model
alpha=ratingmean
userbiases=defaultdict(float)
itembiases=defaultdict(float)
usergamma={}
itemgamma={}
k=2
for u in reviewsperuser:
  usergamma[u]=[random.random()*0.1 - 0.05 for k in range(k)]
for i in reviewsperitem:
  itemgamma[i]=[random.random()*0.1 - 0.05 for k in range(k)]

In [40]:
'''again we need a method to convert from a flat parameter vector(theta)
to our parameters(alpha,betta and gamma)'''
def unpack(theta):
  global alpha
  global userbiases
  global itembiases
  global usergamma
  global itemgamma
  index=0
  alpha=theta[index]
  index+=1
  userbiases=dict(zip(users,theta[index:index+nusers]))
  index+=nusers
  itembiases=dict(zip(items,theta[index:index+nitems]))
  index+=nitems
  for u in users:
    usergamma[u]=theta[index:index+k]
    index+=k
  for i in items:
    itemgamma[i]=theta[index:index+k]

'again we need a method to convert from a flat parameter vector(theta)\nto our parameters(alpha,betta and gamma)'

In [41]:
def inner(x,y):
  return sum([a*b for a,b in zip(x,y)])
def prediction(user,item):
  return alpha+userbiases[user]+itembiases[item]+inner(usergamma[user],itemgamma[item])

In [42]:
def cost(theta,labels,lamb):
  unpack(theta)
  predictions=[prediction(d['customer_id'],d['product_id'])for d in dataset]
  cost=MSE(predictions,labels)
  print("MSE= "+STR(cost))
  for u in users:
    cost+=lamb*userbiases[u]**2
    for k in range(k):
      cost+=lamb*usergamma[u][k]**2
  for i in items:
    cost+=lamb*itembiases[i]**2
    for k in range(k):
      cost+=lamb*itemgamma[i][k]**2
  return(cost)
    
